![CSTR Diagram](../assets/Image_README.png)
# CSTR Simulator

A modular, GUI-integrated simulator for continuous stirred-tank reactors (CSTRs) with recycle loop, temperature optimization, and real industrial case studies.

This notebook was created as a collaborative project for the **EPFL** course _Practical Programming in Chemistry (**CHE-200**)_.

---

## Introduction

youpi

---

## Why Continuous Flow Stirred Reactors?

...

## 🧪 CTSR Simulator – Function Overview

Each key function from the simulator and the Streamlit interface is explained below with its **role**, followed by an **example usage** (code block), in the same visual format as MASSIVEChem notebooks.

---

**🔹 set_parameters(...)**

This method initializes the reactor’s key parameters: volume, temperature, feed flow rate, reactions list, feed composition, recycle ratio, and target product.


In [ ]:
from cstr_simulator.simulator import CSTRSimulator

sim = CSTRSimulator()
sim.set_parameters(volume=5.0, temperature=600, flow_rate=0.3,
                   reactions=[reaction], feed_composition=feed,
                   recycle_ratio=0.2, target_product="C")

**🔹 solve_steady_state(...)**

Solves the steady-state balance of species in the reactor using mass balance and an optimization algorithm.

In [ ]:
results = sim.solve_steady_state()
print(results["concentration"])

**🔹 optimize_temperature(...)**

Performs optimization to find the best operating temperature that maximizes the target product concentration.

In [ ]:
best_temp = sim.optimize_temperature()
print(f"Optimal temperature: {best_temp} K")

**🔹 run_simulation(...)**

Main function that runs the full process simulation from initialization to steady-state computation.

In [ ]:
sim.set_parameters(...)  # first set parameters
sim.run_simulation()

**🔹 load_reaction_data_from_csv(path)**

Utility method in the Streamlit interface to load reaction data from a CSV file.

In [ ]:
from cstr_gui.utils import load_reaction_data_from_csv

reaction_data = load_reaction_data_from_csv("data/reactions.csv")

**🔹 build_simulator_from_user_input(inputs)**

Constructs a CSTRSimulator instance based on values entered in the Streamlit GUI.

In [ ]:
sim = build_simulator_from_user_input(user_inputs)

**🔹 display_results(sim)**

Displays the results using Streamlit’s plotting tools.

In [ ]:
display_results(sim)